## <center>语言模型的系统学习</center>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
gpu_no = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_no
# 定义TensorFlow配置
config = tf.ConfigProto()
# 配置GPU内存分配方式，按需增长，很关键
config.gpu_options.allow_growth = True
# 配置可使用的显存比例
config.gpu_options.per_process_gpu_memory_fraction = 0.3
# 在创建session的时候把config作为参数传进去
sess = tf.Session(config = config)

### 1 词向量
语言模型包括文法语言模型和统计语言模型,一般我们指的是统计语言模型. word embedding 的映射方法是建立在分布假说基础上的,即假设词的语义由其上下文决定,上下文相似的单词,其语义相似. 上一节中,所使用的词向量是 one_hot 编码,由于 one_hot 编码中所有的字都是独立的,所以该语言模型学到的词与词的上下文信息只能存放在网络节点中.

#### (1) 词向量训练
在神经网络训练的词嵌入中, 一般会将所有的 embedding 随机初始化,然后在训练过程中不断更新 embedding 矩阵的值. 对于每一个词与它对应的向量的映射值,在 Tensorflow 中使用了一个叫 $tf.nn.embedding\_lookup$ 的方法来完成. 举例如下:
```python
with tf.device("/cpu:0"):
    embedding = tf.get_variable("embedding",[vocab_size,vec_size])
    inputs = tf.nn.embedding_lookup(embedding,input_data)
```
上面代码事先定义了 embedding 有 vocab_size 个词,每个词的向量个数为 vec_size 个. 最终得到的 inputs 就是输入向量 input_data 映射好的词向量了. 比如 input_data 的形状为 [batch_size,n_word], 那么 inputs 就变成了 [batch_size,n_word,vec_size]

<font color="red">注意</font><br/>
由于词向量定义好之后需要在训练过程中优化,所以 embedding 类型必须是 $tf.Variable$ 并且 $trainable = True$, $embedding\_lookup$ 这个函数目前只支持在 cpu 上运行

#### (2) CBOW & Skip-Gram
训练词向量时,比较常见的组合是 Skip-Gram 和 负采样方法, 这种组合速度快,效果好. 通常来说 <font color="red">CBOW 要比 Skip-Gram 更快</font>
+ CBOW 也是统计语言模型的一种,顾名思义就是根据某个词前面的 $n$ 个词或者前后 $n$ 个连续的词,来计算某个词出现的概率.
+ Skip-Gram 模型与之相反,是根据某个词,然后分别计算它前后出现的某几个词的各个概率.

例如, "我爱人工智能" 对于 CBOW 模型来讲,首先将所有字转换成 one_hot, 然后取出其中的一个字当做输入,将其前面和后面的字分别当做标签,拆分成如下样子:

+ "我",  "爱"
+ "爱",  "我"
+ "爱",  "人"
+ "人",  "爱"
+ "人",  "工"

每一行代表一个样本,第一列代表输入,第二列代表标签.将输入数据送进神经网络,同时将输出与标签计算 loss.进行优化迭代,在整个词库中如果字数特别多,会产生很大的矩阵,影响 softmax 速度.

word2vec 使用基于 Huffman 编码的 Hierarchical softmax 筛选掉了一些不可能的词,然后又使用 negative sampling 再去除一些负养的词, 所以时间复杂度就从 $O(V)$ 变成了 $O(log V)$

#### (3) 在 Tensorflow 中的  word2vec

在 Tensorflow 中提供了几个候选采样函数,用来处理 loss 计算中候选采样的工作,他们按照不同的采样规则被封装成了不同的函数,说明如下:
```python
+ tf.nn.uniform_candidate_sampler: 均匀的采样出类别子集
+ tf.nn.log_uniform_candidate_sampler: 按照 log_uniform (Zipfian) 分布采样, 指只有少数词经常被使用
+ tf.nn.learned_unigram_candidate_sampler: 按照训练数据中出现的类别分布进行采样
+ tf.nn.fixed_unigram_candidate_sampler: 按照用户提供的概率分布进行采样
```
如果不知道样本的分布,可以使用 $tf.nn.learned\_unigram\_candidate\_sampler$. 它的做法是先初始化一个 [0,range_max] 的数组,数组元素初始值为 1, 在训练过程中,碰到一个类别,就将相应的数组元素加 1, 每次按照数组归一化得到的概率进行采样来实现.

<font color="red">注意</font><br/>
在语言相关的任务中,词按照出现频率从大到小排序之后,服从 Zipfian 分布. 一般会先对类别按照出现频率从大到小排序,然后使用 $tf.nn.log\_uniform\_candidate\_sampler$ 函数.

在 TEnsorflow 的 word2vec 实现里,对比目标样本的损失值,计算 softmax,负采样等过程统统封装到了 nce_loss 函数中, 其默认使用的是 $tf.log\_uniform\_candidate\_sampler$ 采样函数,在不指定特殊采样器时,在该函数实现中会把词频越大的词,其编号也定义的越大,<font color='red'>即优先采用词频高的词作为负样本,词频越高越有可能作为负样本</font>, nce_loss 函数配合优化器可以对最后一层的权重进行调优.

#### (4) nce_loss 函数
nce_loss 函数定义如下:
```python
tf.nn.nce_loss(
    weights,
    biases,
    labels,
    inputs,
    num_sampled,
    num_classes,
    num_true=1,
    sampled_values=None,
    remove_accidental_hits=False,
    partition_strategy='mod',
    name='nce_loss')
    
假设输入数据的维度是 K 维,一共有 N 个类,其参数说明如下:
+ weight: shape 为 (N,K) 的权重
+ biases: shape 为 (N) 的偏置
+ inputs: 输入数据,shape 为 (batch_size,K)
+ labels: 标签数据,shape 为 (batch_size,num_true)
+ num_true: 实际的正样本个数
+ num_sampled: 采样出多少个负样本
+ num_classes: 类的个数 N
+ sampled_values: 采样出的负样本,如果是 None, 就会默认的 sampler 去采样,优先采用高频词作为负样本
+ remove_accidental_hits: 如果采样时采样到的负样本刚好是正样本,是否要去掉
+ partition_strategy: 对 weight 进行 embedding_lookup 时并行查表时的策略.embedding_lookup 是在 cpu 里实现的,需要考虑多线程查表时的锁的问题.
```

<font color="red">注意</font><br/>
在 Tensorflow 中还有一个类似于 $nce\_loss$ 的函数, $sampled\_softmax\_loss$, 其用法和 $nce\_loss$ 完全相同,不同之处在于内部实现, $nce\_loss$ 函数可以进行多标签分类问题,即标签之间不互斥,原因在于对其每一个输出的类都链接一个 logistic 二分类, 而 $sampled\_softmax\_loss$ 只能对单个标签分类,即输出的类别是互斥的,原因是其对每个类别的输出放在一起统一做了一个多分类操作.<font color="red">最直观的理解,多标签分类,比如一部电影可以同时被分为动作片和犯罪片.</font>

### 2 用 CBOW 模型训练自己的 Word2Vec

In [2]:
import random
import collections
from collections import Counter
import jieba
from sklearn.manifold import TSNE
import matplotlib as mpl
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['font.family'] = 'STSong'
mpl.rcParams['font.size'] = 20

#### (1) 准备样本,创建数据集

In [3]:
training_file = './data/text/word.txt'

# 处理汉字
def get_ch_words(txt_file):
    words = ""
    with open(txt_file,'rb') as f:
        for word in f:
            words = words + word.decode('utf-8')
    return words

# 将文本分词
def get_segment_word(training_data):
    seg_list = jieba.cut(training_data)
    words = " ".join(seg_list)
    words = words.split()
    words = np.array(words)
    words = np.reshape(words,[-1,])
    return words

# 将统计词频 0 号 位置给 UNK,其余按照词频由高到低排序,UNK 的获取按照预设字典的大小,
# 比如 350,则频次排序靠后小鱼 350 的都视为 UNK
def build_dataset(seg_words,n_words):
    count = [['UNK',-1]]
    count.extend(collections.Counter(seg_words).most_common(n_words-1))
    dictionary = dict()
    for word,_ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in seg_words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dict = dict(zip(dictionary.values(),dictionary.keys()))
    print("count: ")
    print(count)
    return data,count,dictionary,reversed_dict

training_data = get_ch_words(training_file)
print("total words:")
print(len(training_data))

# 分词之后的词列表
seg_words = get_segment_word(training_data)
print("seg_words size:")
print(len(seg_words))
# 获取分词以及对应的词id
training_word,count,dictionary,words = build_dataset(seg_words,100)
vocab_size = len(dictionary)
print("dictionary:")
print(len(dictionary))
print("sample data:")
print(training_word[:10])
print([words[i] for i in training_word[:10]])

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


total words:
206


Loading model cost 0.474 seconds.
Prefix dict has been built succesfully.


seg_words size:
121
count: 
[['UNK', 0], ('，', 23), ('。', 3), ('在', 2), ('枫叶', 2), ('地', 2), ('更加', 2), ('将', 2), ('他', 2), ('与', 2), ('之上', 1), ('仿佛', 1), ('清瘦', 1), ('”', 1), ('荡漾', 1), ('红红', 1), ('书集', 1), ('眼眸', 1), ('十足', 1), ('适中', 1), ('掌声雷动', 1), ('所有', 1), ('简直', 1), ('香', 1), ('树', 1), ('相通', 1), ('熠熠', 1), ('曹老', 1), ('了', 1), ('心灵', 1), ('浅黄', 1), ('愈发', 1), ('新花', 1), ('感觉', 1), ('别开生面', 1), ('花朵', 1), ('“', 1), ('烘焙', 1), ('嫁接', 1), ('三生', 1), ('老树', 1), ('灵魂', 1), ('、', 1), ('精气神', 1), ('正红', 1), ('神韵', 1), ('璀璨', 1), ('强烈', 1), ('震撼', 1), ('四字', 1), ('绚丽', 1), ('有神', 1), ('把', 1), ('想象', 1), ('融化', 1), ('这种', 1), ('丰硕', 1), ('要', 1), ('舞蹈', 1), ('殆尽', 1), ('峭立', 1), ('人生', 1), ('就是', 1), ('开出', 1), ('使', 1), ('经久不息', 1), ('我', 1), ('硬朗', 1), ('红艳', 1), ('蹁跹', 1), ('曹树清', 1), ('拜读', 1), ('之书作', 1), ('明黄', 1), ('扉页', 1), ('身材', 1), ('超乎', 1), ('仙风道骨', 1), ('之花', 1), ('老', 1), ('飘逸', 1), ('老先生', 1), ('枯藤', 1), ('更是', 1), ('三世', 1), ('封面', 1), ('受到', 1), ('溢余', 1), ('鹤然', 

#### (2) 获取批次数据
定义 generate_batch 函数,取一定批次的样本数据.

In [4]:
data_index = 0
def generate_batch(data,batch_size,num_skips,skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2*skip_window
    # 假设这里取 8 个词作为一个 batch,那么每个词都有一个预测目标.
    # 所以 label 的形状应该为 [batch_size,1]
    batch = np.ndarray(shape=(batch_size),dtype = np.int32)
    labels = np.ndarray(shape=(batch_size,1),dtype = np.int32)
    # 每个样本由前 skip_window + 当前 targe + 后 skip_window 组成
    span = 2*skip_window + 1
    buffer = collections.deque(maxlen = span)
    # 判断 data 中的数据是否取完了,如果是,则从头取
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index+span])
    data_index += span
    for i in range(batch_size//num_skips):
        # target 在 buffer 中索引为 skip_window 的位置
        target = skip_window
        target_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in target_to_avoid:
                target = random.randint(0,span-1)
            target_to_avoid.append(target)
            batch[i*num_skips +j] = buffer[skip_window]
            labels[i*num_skips+j,0] = buffer[target]
        if data_index == len(data):
            buffer = data[:span]
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

batch,labels = generate_batch(training_word,batch_size = 8,num_skips=2,skip_window=1)
for i in range(8):
    print(batch[i],words[batch[i]],'->',labels[i,0],words[labels[i,0]])
    

33 感觉 -> 55 这种
33 感觉 -> 1 ，
1 ， -> 33 感觉
1 ， -> 31 愈发
31 愈发 -> 1 ，
31 愈发 -> 5 地
5 地 -> 31 愈发
5 地 -> 86 受到


#### (3) 定义取样参数
下面代码中,每个批次取64个,每个词向量维度为 128, 前后取词窗口为1,num_skips 表示一个 input 生成 2 个标签,nce 中采样的个数为 num_sampled . 接下来是验证模型的相关参数, valid_size 表示 0-words_size/2 中的数取随机不重复的 16 个字来验证模型.

In [5]:
batch_size = 64
embedding_size = 128
skip_window = 1
num_skips = 2
valid_size = 8
valid_window = int(vocab_size/2)
# 在 0 - words_size 中的数取 16 个. 不能重复
valid_examples = np.random.choice(valid_window,valid_size,replace=False)
num_sampled = 64

#### (4) 定义模型变量
初始化图,为输入,标签,验证集定义占位符,定义词嵌入变量 embedding 为每个字定义 128 维的向量,并初始化为 -1到1 之间的均匀分布随机数. $tf.nn.embedding\_lookup$ 是将输入的 train_inputs 转成对应的 128 维向量 embed,定义 nce_loss 需要使用的 nce_weights 和 nce_biases

In [6]:
tf.reset_default_graph()
train_inputs = tf.placeholder(dtype=tf.int32,shape=[batch_size])
train_labels = tf.placeholder(dtype=tf.int32,shape=[batch_size,1])
valid_dataset = tf.constant(dtype=tf.int32,value=valid_examples)

# 在 CPU 上执行
with tf.device('/cpu:0'):
    # 查找 embeddings
    embeddings = tf.Variable(tf.random_uniform([vocab_size,embedding_size],-1.0,1.0))
    embed = tf.nn.embedding_lookup(embeddings,train_inputs)
    
    # 计算 NCE 的 loss 值
    nce_weights = tf.Variable(tf.truncated_normal([vocab_size,embedding_size],
                                                  stddev=1.0/tf.sqrt(np.float32(embedding_size))))
    nce_biases = tf.Variable(tf.zeros([vocab_size]))

#### (5) 定义损失函数和优化器
使用 nce_loss 计算 loss 来保证 softmax 时的运算速度 不被 vocab_size 过大问题所影响, 在使用 nce 中每次会产生 num_sampled(64) 个负样本来参与概率计算.

In [7]:
loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,biases=nce_biases,
                                     labels = train_labels,inputs = embed,
                                     num_sampled=num_sampled,num_classes = vocab_size))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)
# 计算 minibatch examples 和 所有 embedding 的 cosine 相似度
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings),axis=1,keep_dims=True))
normalized_embeddings = embeddings / norm
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings,valid_dataset)
similarity = tf.matmul(valid_embeddings,normalized_embeddings,transpose_b= True)

验证数据做了特殊的处理,将 embeddings 中每个词对应的向量进行平方和再开方得到 norm,然后将 embedding 与 norm 相除得到 normalized_embeddings. 当使用 embedding_lookup 获得自己对应 normalized_embedding 中的向量 valid_embeddings 时,将该向量与转置后的 normalized_embeddings 相乘得到每个词的 similarity. 这个过程实现了一个向量间夹角余弦的计算

#### (6) 夹角余弦
余弦定理: 给定三角形的三条边 $a,b,c$, 对应的是三个角分别为 $A,B,C$ 则夹角 $A$ 的余弦为:
<br/><br/>
$$cos A = \frac{b^2+c^2-a^2}{2bc}$$
<br/>
如果将 $b$ 和 $c$ 看成两个向量,则上述公式等价为:
<br/><br/>
$$cos A = \frac{<\boldsymbol{b},\boldsymbol{c}>}{|\boldsymbol{b}|\cdot|\boldsymbol{c}|}$$
<br/>
分母表示两个向量的长度,分子表示两个向量的内积,假设在二维空间, $A(x_1,y_1)$ 与向量 $B(x_2,y_2)$ 的夹角余弦公式为:
<br/><br/>
$$cos \theta = \frac{x_1x_2+y_1y_2}{\sqrt{x_1^2+y_1^2}\sqrt{x_2^2+y_2^2}}$$
<br/>
再扩展到两个 $n$ 维样本点, $a(x_{11},x_{12},...,x_{1n})$ 和 $b(x_{21},x_{22},...,x_{2n})$ 的夹角余弦的公式:
<br/><br/>
$$cos \theta = \frac{x_{11}x_{21}+x_{12}x_{22}+...}{\sqrt{x_{11}^2+x_{21}^2+...}\sqrt{x_{12}^2+x_{22}^2+...}}$$
<br/>
那么前面的代码表示 ***norm*** 代表每一个词对应向量的长度矩阵:
$$
\begin{equation}
\boldsymbol{norm} = \left\{
             \begin{array}{lr}
             \sqrt{x_{11}^2+x_{12}^2+...} &  \\
             \sqrt{x_{21}^2+x_{22}^2+...} & \\
             \sqrt{x_{31}^2+x_{32}^2+...} &  
             \end{array}
\right\}
\end{equation}
$$
<br/>
normalized_embeddings 表示的意思是向量除以自己的模,即单位向量,它可以确定向量的方向. 很显然, similarity 就是 valid_dataset 中对应的单位向量 valid_embedding 与整个词嵌入字典中单位向量的夹角余弦.

如图所示,算了这么多夹角余弦的目的就是为了衡量两个 $n$ 维向量间的相似程度. 当 $cos \theta$ 为1是, 表明夹角为 0, 即两个向量的方向完全一样. 所以当 $cos \theta$ 的值越小,表明两个向量方向越不一样,相似度越低.
<center><img src="./img/12/1.png" width="600"/></center>


#### (7) 训练模型
将验证数据输入到模型中,找出与其最相近的词,这里使用了一个 argsort 函数,是将数组中的值从小到大排序后,返回每个值对应的索引,在使用 argsort 函数之前,将 sim 取负,得到的就是从大到小排列的结果了, sim 就是当前词与整个词典中每个词的夹角余弦.

In [8]:
num_steps = 30001
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    avg_loss = 0.0
    for step in range(num_steps):
        batch_inputs,batch_labels = generate_batch(training_word,batch_size,num_skips,skip_window)
        feed_dict = {train_inputs:batch_inputs,train_labels:batch_labels}
        _,loss_val = sess.run([optimizer,loss],feed_dict=feed_dict)
        avg_loss += loss_val
        

        if step % 2000 == 0:
            if step > 0:
                avg_loss /=2000
            print("step: "+ str(step)+" avg_loss: "+ str(avg_loss))
            avg_loss = 0.0

    sim = similarity.eval(session=sess)
    for i in range(valid_size):
        valid_word = words[valid_examples[i]]
        top_k = 5
        # argsort 函数返回的是数组值从小到大的索引值
        nearest = (-sim[i,:]).argsort()[1:top_k+1]
        log_str = "Nearest to %s:" % valid_word
        for k in range(top_k):
            close_word = words[nearest[k]]
            log_str = '%s,%s' % (log_str,close_word)
        print(log_str)

step: 0 avg_loss: 55.8177604675293
step: 2000 avg_loss: 3.3676299152374267
step: 4000 avg_loss: 2.290874759078026
step: 6000 avg_loss: 2.2636764889359475
step: 8000 avg_loss: 2.250925530254841
step: 10000 avg_loss: 2.248628088414669
step: 12000 avg_loss: 2.2457841641306877
step: 14000 avg_loss: 2.243143747806549
step: 16000 avg_loss: 2.238080579459667
step: 18000 avg_loss: 2.2359620010256767
step: 20000 avg_loss: 2.2387845514416695
step: 22000 avg_loss: 2.2396301223039625
step: 24000 avg_loss: 2.233453496694565
step: 26000 avg_loss: 2.234935766160488
step: 28000 avg_loss: 2.233838494062424
step: 30000 avg_loss: 2.2317771494984626
Nearest to 枫叶:,灵魂,有神,经久不息,舞蹈,开出
Nearest to 他:,将,红艳,相通,三生,别开生面
Nearest to 熠熠:,新花,要,树,融化,”
Nearest to 简直:,感觉,更是,舞蹈,溢余,遍
Nearest to 别开生面:,清瘦,感觉,仙风道骨,红红,硬朗
Nearest to 香:,地,烘焙,精气神,相通,掌声雷动
Nearest to 在:,人生,想象,鹤然,三世,掌声雷动
Nearest to “:,蹁跹,。,、,更是,新花
